# Lab 2 Report

This lab is about developing a USB Mouse for PSOC-5 using a potentiometer joystick and an i2C joystick from SparkFun. The purpose of this lab is to introduce interfacing a device over USB with a desktop or laptop. One joystick is purely a potentiometer joystick that needs can be used by reading the voltage output with an ADC. The other joystick pre-processes the joystick information using an onboard ATtiny and sends the data over I2C to a connected device. 

Problems for Potentiometer and I2C combined:

UART would not connect.                         Solution: remade project after installing KIT for Psoc 5

Not showing up on COM port.                     Solution: Hard reset the board by holding the reset switch at least 5 seconds.

ADC values were not displaying.                 Solution: sprintf on the psoc does not display floats properly. Typecast the value to int
 
No x or y data being put in mouse data array.   Solution: Used double temp variable to calculate value then put that in mouse array

Tweaking values for usability. Normalized the adc output to between -40 and 40, which was too high for usable cursor movement. Lowered it incrementally until I settled on -10 to 10. 
The joystick did not need a software deadzone as the physical deadzone was plenty. 

The raw ADC output is 7 bits unsigned. To normalize it, divide by 2^7 = 128 then multiply by the max value desired. In my case, I used 10. To center the data on 0 when the joystick was neutral, subtracted 10. 

Mouse circuit

The potentiometer mouse only used the ADC channels and power to read the joystick position, shown in the PSoC Creator schematic ![here](Images\PSoCPOTSchematic.png)

The I2C was connected through a level shifter for 5V to 3.3V signal shifting. In PSoC Creator, the I2C module is connected to SDA and SCL pins and reads the data from the I2C bus. Schematic: ![here](Images\PSoCI2CSchematic.png). 

The I2C protocol built into the joystick sends bytes of data at a time, corresponding to different data points. the 8th byte of data corresponds to the value of the pushbutton on the joystick, for example. In the main.c for the project, the PSoC reads data from the I2C bus and interprets it before sending the data over USB to the connected device (computer). It also reads ADC data from the potentiometer joystick

Code [here](I2C_mouse\I2C_Mouse\I2C_Mouse.cydsn\main.c)

The I2C joystick used the I2C bus and a level shifter to interact with the 3.3V signals from the sparkfun joystick

![I2C and ADC joystick setup](Images/i2cAndADCmouse.png)

Both joysticks functioning

The potentiometer joystick uses the ADC on the PSOC to read the voltage values on each of the joystick axes, then converts that into values that are sent through USB to the connected computer. The joystick is able to move the cursor horizontally and vertically as well as left click. 

The I2C joystick, also connected to the PSOC, passed signals through a level shifter to control the cursor in the same way as the potentiometer joystick. 

Demo video in [demo](Images/BothJoysticks_demo.mp4)

I started the project having only the ADC framework from Lab 1 to read the potentiometer joystick voltages. I also made a copy of the included example project for a USB Mouse, which moved the cursor in a square path to demo USB functionality. 

imported code from PSoC Creator


    #include <project.h>
    #include <stdio.h>

    static int8 Mouse_Data[3] = {0,0,0}; /* [0] = Buttons, [1] = X-Axis, [2] = Y-Axis */


    int main()
    {
        CYGlobalIntEnable;                        		/* Enable Global Interrupts */
        USBFS_1_Start(0, USBFS_1_DWR_VDDD_OPERATION);   /* Start USBFS Operation/device 0 and with 5V operation */ 	
        while(!USBFS_1_bGetConfiguration()){};      		/* Wait for Device to enumerate */
        USBFS_1_LoadInEP(1, (uint8 *)Mouse_Data, 3); 	/* Loads an inital value into EP1 and sends it out to the PC */
        
        ADC_Start();
        I2C_Start();
        UART_Start();
        UART_PutString("Open\n");
        char buffer[100];
        uint8 i2cdata[10];//10 bytes of data - bytes 3+4, 5+6, and 7 correspond to mousex, mousey, and mousebutton
        double temp;
        
        
        for(;;)
        {
            while(!USBFS_1_bGetEPAckState(1));  			/* Wait for ACK before loading data */
            USBFS_1_LoadInEP(1, (uint8 *)Mouse_Data, 3); 	/* Load latest mouse data into EP1 and send to PC */
                    
            I2C_MasterReadBuf(0x20, i2cdata,10, I2C_MODE_COMPLETE_XFER);
            while(I2C_MasterStatus()!=I2C_MSTAT_RD_CMPLT); //wait for i2c to finish
                        
            Mouse_Data[0] = (1-i2cdata[7]) | (!SW_Read());
            Mouse_Data[1] = ((i2cdata[3]<<8 | i2cdata[4])-31168)/(float)3000;
            Mouse_Data[2] = ((i2cdata[5]<<8 | i2cdata[6])-32768)/(float)3000;
            
            sprintf(buffer, "0:%i\t 1:%i\t 2:%i\t\n",
                Mouse_Data[0], Mouse_Data[1], Mouse_Data[2]);
            
            while(!USBFS_1_bGetEPAckState(1));  			/* Wait for ACK before loading data */
            USBFS_1_LoadInEP(1, (uint8 *)Mouse_Data, 3); 	/* Load latest mouse data into EP1 and send to PC */
            
            for(int channel = 0; channel < 2; channel++){
                ADC_StartConvert();
                ADC_IsEndConversion(ADC_WAIT_FOR_RESULT); 
                temp = (ADC_GetResult16(channel)/128.0)*10.0-10; 
                
                Mouse_Data[channel+1] = temp;
                sprintf(buffer, "channel:%i \t temp:%u \t", channel, (unsigned)temp);
                UART_PutString(buffer);
            
            }
        
            UART_PutString(buffer);
                
            CyDelay(1);
        }
    }
    /* End of File */